In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'
#This script detects ARs and the sorts EP days into weather types

In [ ]:
#Taiwan bounds: 0-55 N,80-160 E

ds=xr.open_dataset(dir+'ls_extreme_rain_taiwan.nc')
p=ds['large_scale_extreme_rain_all']#total rainfall on ER days
threshold=ds['threshold_99']
dates=ds.large_scale_extreme_days
print(dates)

ds0=xr.open_dataset(dir+'ivt_basics_taiwan_95.nc')#DO NOT CHANGE

ds1=xr.open_dataset(dir+'ivt_basics_taiwan_99.nc')
ds1['longitude']=ds0.longitude
ivt_threshold=ds1['ivt_threshold']
ivt_mag=ds1['ivt_mag']
print(ivt_mag)
ivtx=ds1['ivtx']
ivty=ds1['ivty']
lons_full=ivt_mag.longitude.values
lats_full=ivt_mag.latitude.values

ds3=xr.open_dataset(dir+'taiwan_uv_850_extreme_days.nc')
u_850=ds3.u_850
v_850=ds3.v_850
ds4=xr.open_dataset(dir+'taiwan_slp_extreme_days.nc')
slp=ds4.slp/100.
print(slp)
print(slp.time.values)

ds6=xr.open_dataset(dir+'IBTrACS.WP.v04r00.nc')
#print(ds)
lat_tc=ds6.lat
lon_tc=ds6.lon
#nature_tc=ds6.nature
ds7=xr.open_dataset(dir+'ibtracs_wp_time.nc')
years_tc=ds7.years
months_tc=ds7.months
days_tc=ds7.days
hours_tc=ds7.hours
nature_tc=ds7.nature#1 if tropical, 0 if not

In [ ]:
dates_unique=[]
for i in range(10):
    year=2010+i
    date_start=dt.datetime(year,5,14,18)
    date_range=[date_start+dt.timedelta(days=x) for x in range(47)]
    if i==0:
        dates_unique=date_range
    else:
        dates_unique=dates_unique+date_range
print(dates_unique)

In [ ]:
#Part 1 - AR detection
#dates_unique1=dates_unique[0:2]
for d in range(len(dates_unique)):
#for d in range(2):
    date=dates_unique[d]#date is at 6Z automatically.  IVT goes from 6Z on day to 6Z on next day
    date_range=[date+dt.timedelta(hours=6*x) for x in range(4)]
    date_range=pd.DatetimeIndex(date_range).values#date range for IVT selection
    for date in date_range:
        print(date)
        date=pd.to_datetime(date)
        year=date.year
        month=date.month
        day=date.day
        hour=date.hour
        ivt_mag_current=ivt_mag.sel(time=date)
        ivt_extreme=ivt_mag_current.where(ivt_mag_current>=ivt_threshold)
        ivt_extreme_full=ivt_extreme
        ivt_ones=ivt_extreme/ivt_extreme
        ivt_ones=ivt_ones.fillna(0)
        structure = np.ones((3, 3), dtype=np.int)          
        labeled_full,ncomponents_full=label(ivt_ones,structure) 
        labeled_xr_full=xr.zeros_like(ivt_extreme)
        labeled_xr_full[:,:]=labeled_full
        if d==0 and date==date_range[0]:
            labeled_ivt_points=labeled_xr_full
        else:
            labeled_ivt_points=xr.concat([labeled_ivt_points,labeled_xr_full],dim='time')
            
lats_tc_list=lat_tc.values.ravel()
lons_tc_list=lon_tc.values.ravel()
years_tc_list=years_tc.values.ravel()
months_tc_list=months_tc.values.ravel()
days_tc_list=days_tc.values.ravel()
hours_tc_list=hours_tc.values.ravel()
nature_tc_list=nature_tc.values.ravel()

zipped_tc=list(zip(lats_tc_list,lons_tc_list,years_tc_list,months_tc_list,days_tc_list,hours_tc_list,nature_tc_list))
nonzero_tc=[x for x in zipped_tc if x[0]>=0]

lat_array=xr.zeros_like(ivt_mag[0,:,:])
lon_array=xr.zeros_like(ivt_mag[0,:,:])
for i in range(ivt_mag.longitude.size):
    lat_array[:,i]=ivt_mag.latitude.values
for i in range(ivt_mag.latitude.size):
    lon_array[i,:]=ivt_mag.longitude.values
    
lats_full=ivt_mag.latitude.values
lons_full=ivt_mag.longitude.values
lon_array_list=lon_array.values.ravel()
lat_array_list=lat_array.values.ravel()

lats_xr=lat_array.copy()[0::2,0]
lons_xr=lon_array.copy()[0,0::2]
lats_array_xr=lat_array.copy()[0::2,0::2]
lons_array_xr=lon_array.copy()[0::2,0::2]
lats_xr_flat=lats_array_xr.values.ravel()
lons_xr_flat=lons_array_xr.values.ravel()

    
zipped_latlon=list(zip(lat_array_list,lon_array_list))

for d in range(len(dates_unique)):
#for d in range(2):
    print(d)
    date=dates_unique[d]#date is at 6Z automatically.  IVT goes from 6Z on day to 6Z on next day
    date_range=[date+dt.timedelta(hours=6*x) for x in range(4)]
    date_range=pd.DatetimeIndex(date_range).values#date range for IVT selection
    for date in date_range:
        print(date)
        date=pd.to_datetime(date)
        year=date.year
        month=date.month
        day=date.day
        hour=date.hour
        ivt_mag_current=ivt_mag.sel(time=date)
        ivtx_current=ivtx.sel(time=date)
        ivty_current=ivty.sel(time=date)

        ivt_extreme=ivt_mag_current.where(ivt_mag_current>=ivt_threshold)
        ivt_extreme_full=ivt_extreme
        ivt_ones=ivt_extreme/ivt_extreme
        ivt_ones=ivt_ones.fillna(0)
        
        ###Now load TC data and find regions within a certain radius (currently 500 km) of TC centers
        
        check_lat_tc_trop_list=[x[0] for x in nonzero_tc if x[2]==year and x[3]==month and x[4]==day and x[5]==hour and x[6]==1]
        check_lat_tc_nontrop_list=[x[0] for x in nonzero_tc if x[2]==year and x[3]==month and x[4]==day and x[5]==hour and x[6]==0]
        check_lon_tc_trop_list=[x[1] for x in nonzero_tc if x[2]==year and x[3]==month and x[4]==day and x[5]==hour and x[6]==1]
        check_lon_tc_nontrop_list=[x[1] for x in nonzero_tc if x[2]==year and x[3]==month and x[4]==day and x[5]==hour and x[6]==0]

        #print('A')
        
        if len(check_lat_tc_trop_list)>0 and len(check_lon_tc_trop_list)>0:
            #print('TROP')
            for i in range(len(check_lat_tc_trop_list)):
                lat_tc_single=check_lat_tc_trop_list[i]
                lon_tc_single=check_lon_tc_trop_list[i]
                first_elim=[x for x in zipped_latlon if abs(x[0]-lat_tc_single)<5.0 or abs(x[1]-lon_tc_single)<10.0]
                select_ball_trop=[x for x in first_elim if geopy.distance.distance((x[0],x[1]),(lat_tc_single,lon_tc_single)).km<500.]    
                if i==0:
                    all_trop_points=select_ball_trop
                else:
                    all_trop_points=list(set(all_trop_points+select_ball_trop))
            all_trop_pairs=[(x[0],x[1]) for x in all_trop_points]
                    
        if len(check_lat_tc_nontrop_list)>0 and len(check_lon_tc_nontrop_list)>0:
            #print('NONTROP')
            for i in range(len(check_lat_tc_nontrop_list)):
                lat_tc_single=check_lat_tc_nontrop_list[i]
                lon_tc_single=check_lon_tc_nontrop_list[i]
                first_elim=[x for x in zipped_latlon if abs(x[0]-lat_tc_single)<5.0 or abs(x[1]-lon_tc_single)<10.0]
                select_ball_nontrop=[x for x in first_elim if geopy.distance.distance((x[0],x[1]),(lat_tc_single,lon_tc_single)).km<500.]
                if i==0:
                    all_nontrop_points=select_ball_nontrop
                else:
                    all_nontrop_points=list(set(all_nontrop_points+select_ball_nontrop))
            all_nontrop_pairs=[(x[0],x[1]) for x in all_nontrop_points]
        
        labeled_tc_trop=xr.zeros_like(ivt_mag_current)
        labeled_tc_nontrop=xr.zeros_like(ivt_mag_current)
        for i in range(len(lats_full)):
            for j in range(len(lons_full)):
                if len(check_lat_tc_trop_list)>0:
                    if (lats_full[i],lons_full[j]) in all_trop_pairs:
                        labeled_tc_trop[i,j]=1
                if len(check_lat_tc_nontrop_list)>0:
                    if (lats_full[i],lons_full[j]) in all_nontrop_pairs:
                        labeled_tc_nontrop[i,j]=1
                    
        #print('B')
        #print(labeled_tc_trop.max().values)
        
        structure = np.ones((3, 3), dtype=np.int)          
        ivt_ones_ar=ivt_ones.where(labeled_tc_trop==0).fillna(0)
        labeled,ncomponents=label(ivt_ones_ar,structure)

        #AR length is greatest distance between two points in feature
        labeled_xr=xr.zeros_like(ivt_extreme)
        labeled_xr[:,:]=labeled

        endpoint1_list=[]
        endpoint2_list=[]
        distance_max_list=[]
        mean_width_list=[]
        north_component_list=[]
        ivt_mag_current=ivt_mag_current[0::2,0::2]
        ivtx_current=ivtx_current[0::2,0::2]
        ivty_current=ivty_current[0::2,0::2]
        labeled_xr_full=labeled_xr
        labeled_xr=labeled_xr[0::2,0::2]
        labeled_xr_flat=labeled_xr.values.ravel()
        
        #print('C')
        
        nums_list=[]
        points_boundary_list=[]
        lat_first_list=[]
        lon_first_list=[]
        boundaries_xr=xr.zeros_like(labeled_xr)
        area_sum_list=[]

        for i in range(ncomponents+1):
            select_component=labeled_xr.where(labeled_xr==i)
            select_component=select_component-select_component
            select_component=select_component+1
            num_connected=select_component.sum().values
            nums_list.append(num_connected)
            
        zipped_num=list(zip(np.arange(ncomponents+1),nums_list))
        sorted_num=sorted(zipped_num,key=lambda x:x[1])
        largest_num=sorted_num[-1][0]
        zero_indices=[x[0] for x in sorted_num if x[1]==0]
        components_num_list=[x for x in np.arange(ncomponents+1) if x!=largest_num and x not in zero_indices]
        
        ivt_extreme=ivt_extreme[0::2,0::2]
        
        ar_timestep=xr.zeros_like(labeled_xr_full)
        artc_timestep=xr.zeros_like(labeled_xr_full)
        
        labeled_full_select=labeled_ivt_points.sel(time=date)
        
        for i in components_num_list:
            labeled_test=labeled_xr.where(labeled_xr==i)
            set_number_subset=labeled_full_select.where(labeled_xr_full==i)
            set_number_values=set_number_subset.values.flatten().tolist()
            set_number_values=[x for x in set_number_values if x>0]
            set_number_value=list(set(set_number_values))
            set_number_value=set_number_value[0]
            corresponding_full_label=labeled_full_select.where(labeled_full_select==set_number_value)
            corresponding_full_label=corresponding_full_label/corresponding_full_label
            
            distance_max=0
            labeled_select=labeled_xr.where(labeled_xr==i)
            labeled_select=labeled_select/labeled_select
            labeled_sum=labeled_select.sum().values
            
            labeled_full_notc=labeled_full_select.where(labeled_tc_trop==0).fillna(0)
            
            lats_where=lats_xr.where(labeled_xr==i)
            lons_where=lons_xr.where(labeled_xr==i)
            min_lat=lats_where.min(skipna=True).values
            max_lat=lats_where.max(skipna=True).values
            min_lon=lons_where.min(skipna=True).values
            max_lon=lons_where.max(skipna=True).values

            #print('D')
            if labeled_select.sum(skipna=True).values>0:
                distance_hyp=geopy.distance.geodesic((min_lat,min_lon),(max_lat,max_lon)).km
                if distance_hyp>=2000:#filter out features that could not possibly be long enough
                    proceed='no'
                    ivtx_extreme=ivtx_current.where(labeled_xr==i)
                    ivty_extreme=ivty_current.where(labeled_xr==i)
                    ivtx_mean=ivtx_extreme.mean(dim=('latitude','longitude'),skipna=True).values
                    ivty_mean=ivty_extreme.mean(dim=('latitude','longitude'),skipna=True).values

                    north_component_list.append(ivty_mean)
                    area_sum=0
                    #print('E')
                    
                    zipped_label=list(zip(lats_xr_flat,lons_xr_flat,labeled_xr_flat))
                    select_component_lat=[x[0] for x in zipped_label if x[2]==i]
                    select_component_lon=[x[1] for x in zipped_label if x[2]==i]
                    areas=[geopy.distance.distance((x,y),(x,y+0.25)).km*geopy.distance.distance((x,y),(x+0.25,y)).km for x,y in zip(select_component_lat,select_component_lon)]
                    area_sum=float(sum(areas))
                    
                    necessary_length=area_sum/1000.
                    #print(necessary_length)
                    min_lat=min(select_component_lat)
                    max_lat=max(select_component_lat)
                    min_lon=min(select_component_lon)
                    max_lon=max(select_component_lon)
                    
                    zipped_coords_1=list(zip(select_component_lat,select_component_lon))
                    
                    min_lat_select=[x for x in zipped_coords_1 if x[0]==min_lat]
                    sorted_lat=sorted(min_lat_select,key=lambda x:x[1])
                    minlatminlon=sorted_lat[0]
                    
                    max_lat_select=[x for x in zipped_coords_1 if x[0]==max_lat]
                    sorted_lat=sorted(max_lat_select,key=lambda x:x[1])
                    maxlatmaxlon=sorted_lat[-1]
                    
                    min_lon_select=[x for x in zipped_coords_1 if x[1]==min_lon]
                    sorted_lon=sorted(min_lon_select,key=lambda x:x[0])
                    minlonminlat=sorted_lon[0]
                    
                    max_lon_select=[x for x in zipped_coords_1 if x[1]==max_lon]
                    sorted_lon=sorted(max_lon_select,key=lambda x:x[0])
                    maxlonmaxlat=sorted_lon[-1]
                    
                    
                    dist_1=geopy.distance.distance((minlatminlon[0],minlatminlon[1]),(maxlatmaxlon[0],maxlatmaxlon[1])).km
                    dist_2=geopy.distance.distance((minlonminlat[0],minlonminlat[1]),(maxlonmaxlat[0],maxlonmaxlat[1])).km
                    
                    #print(dist_1,dist_2)
                    if dist_1>=necessary_length or dist_2>=necessary_length:
                        if dist_1>=2000. or dist_2>=2000.:
                            proceed='yes'
                            #print('YES')
                    
                    else:
                        for j in range(len(select_component_lat)-1):
                            lat_1=select_component_lat[j]
                            lon_1=select_component_lon[j]
                            for k in range(j+1,len(select_component_lat)):
                                lat_2=select_component_lat[k]
                                lon_2=select_component_lon[k]
                                distance=geopy.distance.distance((lat_1,lon_1),(lat_2,lon_2)).km
                                if distance>distance_max:
                                    distance_max=distance
                            
                        #print('F')
                        if distance_max>=necessary_length and distance_max>=2000.:
                            proceed='yes'
                    
                    north_component=ivty_mean
                    if proceed=='no':# or north_component<50:
                        pass
                    else:
                        #print('G')
                        corresponding_full_label_ar=corresponding_full_label.where(labeled_xr_full==i).fillna(0)
                        corresponding_full_label_tc=corresponding_full_label.where(labeled_tc_trop==1).fillna(0)
                        corresponding_full_label_nontc=corresponding_full_label.where(labeled_tc_nontrop==1).fillna(0)
                        corresponding_full_label_tc_flat=corresponding_full_label_tc.values.ravel()
                        corresponding_full_label_nontc_flat=corresponding_full_label_nontc.values.ravel()
                        #first: region of high IVT component that is AR-qualified
                        #next: region of high IVT component that is within TC shield
                        #Can these be disconnected?  No, because there would have to be non-AR extreme IVT between TC and AR; but that would be connected to AR, so would belong to AR
                        
                        zipped_label=list(zip(lat_array_list,lon_array_list,corresponding_full_label_tc_flat,corresponding_full_label_nontc_flat))
                        select_component_lat=[x[0] for x in zipped_label if x[2]==i]
                        select_component_lon=[x[1] for x in zipped_label if x[2]==i]
                        areas=[geopy.distance.distance((x,y),(x,y+0.25)).km*geopy.distance.distance((x,y),(x+0.25,y)).km for x,y in zip(select_component_lat,select_component_lon)]
                        area_sum_tc=float(sum(areas))
                    
                        area_ratio=area_sum_tc/area_sum
                        if area_ratio>=1./3.:
                            labeled_full=labeled_xr_full.where(labeled_xr_full==i).fillna(0)
                            artc_timestep=artc_timestep+labeled_full
                        else:
                            labeled_full=labeled_xr_full.where(labeled_xr_full==i).fillna(0)
                            ar_timestep=ar_timestep+labeled_full
                    
        ar_timestep=ar_timestep.where(ar_timestep>0)
        ar_timestep=ar_timestep/ar_timestep
        
        artc_timestep=artc_timestep.where(artc_timestep>0)
        artc_timestep=artc_timestep/artc_timestep
        #print('H')
        #sys.exit()
        if d==0 and date==date_range[0]:
            ar_points=ar_timestep
            artc_points=artc_timestep
            tc_points_trop=labeled_tc_trop
            tc_points_nontrop=labeled_tc_nontrop
        else:
            ar_points=xr.concat([ar_points,ar_timestep],dim='time')
            artc_points=xr.concat([artc_points,artc_timestep],dim='time')
            tc_points_trop=xr.concat([tc_points_trop,labeled_tc_trop],dim='time')
            tc_points_nontrop=xr.concat([tc_points_nontrop,labeled_tc_nontrop],dim='time')

            
dk=xr.Dataset()

dk['ar']=(('time','latitude','longitude'),ar_points)
dk['artc']=(('time','latitude','longitude'),artc_points)
dk['tc_trop']=(('time','latitude','longitude'),tc_points_trop)
dk['tc_nontrop']=(('time','latitude','longitude'),tc_points_nontrop)
dk['labeled']=(('time','latitude','longitude'),labeled_ivt_points)

dk.coords['time']=ar_points.time
dk.coords['latitude']=ar_points.latitude
dk.coords['longitude']=ar_points.longitude
#A->B, B->C, G->H
try:
    os.remove(dir+'taiwan_ar_points_test_var95_ALL_2010_2019.nc')
except OSError:
    pass
dk.to_netcdf(dir+'taiwan_ar_points_test_var95_ALL_2010_2019.nc',mode='w',format='NETCDF4')
#original = _test
#area condition for extended TC moisture flux = _test_elim1

In [ ]:
ds5=xr.open_dataset(dir+'taiwan_ar_points_test_var99.nc')
print(ds5)
ar=ds5.ar
tc_trop=ds5.tc_trop
tc_nontrop=ds5.tc_nontrop
artc=ds5.artc
labeled=ds5['labeled']
print(ar.time)

In [ ]:
#part 2: weather typing
#time_step=50
#ivt_threshold=790.4823648885298
import matplotlib as mpl
month_dict={'1':'January','2':'February','3':'March','4':'April','5':'May','6':'June','7':'July','8':'August','9':'September','10':'October','11':'November','12':'December'}    
ar_yesno=[]
ivt_yesno=[]
artc_yesno=[]
tc_yesno=[]
other_yesno=[]
tc_extended_ivt_yesno=[]
tc_linked_ar_yesno=[]
tc_linked_ivt_yesno=[]
ivt_tc_remnants_yesno=[]
ar_tc_remnant_combo_yesno=[]
tc_remnant_linked_ar_yesno=[]
tc_remnants_yesno=[]
tc_ar_combo_yesno=[]
count_fig=0
lats2=ds1.latitude.values.tolist()
lons2=ds1.longitude.values.tolist()
for d in range(len(dates)):
#for d in range(4):
    print(d)
    date=dates[d].values#date is at 6Z automatically.  IVT goes from 18Z on previous day to 18Z on current day
    date=pd.to_datetime(date)
    date_start=date+pd.Timedelta(hours=-12)
    date_range=[date_start+dt.timedelta(hours=6*x) for x in range(4)]
    date_range=pd.DatetimeIndex(date_range).values#date range for IVT, AR, EP comparison
    
    labeled_four=labeled.sel(time=date_range)
    points_closest_list=[]
    four_labeled_rain=xr.zeros_like(labeled_four)
    p_select=p[d,:,:]
    p_select_extreme=p_select.where(p_select>=threshold)
    lats_extreme_list=[]
    lons_extreme_list=[]
    obs_extreme_list=[]
    for i in range(p.lat.size):
        for j in range(p.lon.size):
            if p_select_extreme[i,j].values>=threshold[i,j].values:
                lat=p.lat.values[i]
                lon=p.lon.values[j]
                obs_1=p_select.values[i,j]
                obs_extreme_list.append(obs_1)
                lats_extreme_list.append(lat)
                lons_extreme_list.append(lon)
                int_lat=int(lat)
                int_lon=int(lon)
                lat_range=[x for x in lats2 if int_lat<=lat<=int_lat+1]
                lon_range=[x for x in lons2 if int_lon<=lon<=int_lon+1]
                dists_lat=[abs(x-lat) for x in lat_range]
                dists_lon=[abs(x-lon) for x in lon_range]
                zipped_lat=list(zip(np.arange(len(lat_range)),dists_lat))
                zipped_lon=list(zip(np.arange(len(lon_range)),dists_lon))
                lat_sorted=sorted(zipped_lat,key=lambda x:x[1])
                lon_sorted=sorted(zipped_lon,key=lambda x:x[1])
                closest_index_lat=lat_sorted[0][0]
                closest_index_lon=lon_sorted[0][0]
                lat_closest=lat_range[closest_index_lat]
                lon_closest=lon_range[closest_index_lon]
                point_closest=(lon_closest,lat_closest)
                if point_closest not in points_closest_list:
                    points_closest_list.append(point_closest)  
                    for i in range(4):
                         four_labeled_rain[i,closest_index_lat-2:closest_index_lat+3,closest_index_lon-2:closest_index_lon+3]=labeled_four[i,closest_index_lat-2:closest_index_lat+3,closest_index_lon-2:closest_index_lon+3].values


    if 1==1:
        ivt_mag_date_four=ivt_mag.sel(time=date_range)
        ivt_mag_date=ivt_mag_date_four.max(dim='time')
        lons_closest=[x[0] for x in points_closest_list]
        lats_closest=[x[1] for x in points_closest_list]
        extreme_ivt_date=ivt_mag_date.where(ivt_mag_date>=ivt_threshold)
        extreme_ivt_value_list=[]
        extreme_ivt_lat_list=[]
        extreme_ivt_lon_list=[]
        for i in range(len(lons_closest)):
            lon=lons_closest[i]
            lat=lats_closest[i]
            ivt_mag_value=ivt_mag_date.sel(latitude=np.arange(lat-2*0.25,lat+3*0.25,0.25),longitude=np.arange(lon-2*0.25,lon+3*0.25,0.25)).max().values
            thresh_value=ivt_threshold.sel(latitude=lat,longitude=lon).values.tolist()
            #thresh_value=ivt_threshold
            if ivt_mag_value>=thresh_value:
                extreme_ivt_value_list.append(ivt_mag_value)
                extreme_ivt_lat_list.append(lat)
                extreme_ivt_lon_list.append(lon)
                
        ar_date_four=ar.sel(time=date_range)
        ar_max_four=ar_date_four.max(dim='time')
        lons_closest=[x[0] for x in points_closest_list]
        lats_closest=[x[1] for x in points_closest_list]
        ar_lat_list=[]
        ar_lon_list=[]
        for i in range(len(lons_closest)):
            lon=lons_closest[i]
            lat=lats_closest[i]
            ar_square=ar_max_four.sel(latitude=np.arange(lat-2*0.25,lat+3*0.25,0.25),longitude=np.arange(lon-2*0.25,lon+3*0.25,0.25))
            ar_value=ar_square.max().values
            if ar_value==1:
                ar_lat_list.append(lat)
                ar_lon_list.append(lon)
        tc_trop_lat_list=[]
        tc_trop_lon_list=[]
        tc_trop_lat_list_day=[]
        tc_trop_lon_list_day=[]
        tc_nontrop_lat_list=[]
        tc_nontrop_lon_list=[]
        tc_nontrop_lat_list_day=[]
        tc_nontrop_lon_list_day=[]

    tc_trop_linked_ivt='no'
    tc_trop_linked_ar='no'
    tc_nontrop_linked_ivt='no'
    tc_nontrop_linked_ar='no'
    for date in date_range:
        date=pd.to_datetime(date)
        year=date.year
        month=date.month
        day=date.day
        hour=date.hour
        ar_select=ar_date_four.sel(time=date)
        labeled_select=labeled.sel(time=date)
        tc_trop_select=tc_trop.sel(time=date)
        tc_nontrop_select=tc_nontrop.sel(time=date)
        
        labeled_tc_trop_check=labeled_select.where(tc_trop_select==1)
        labeled_tc_trop_check=labeled_tc_trop_check.where(labeled_tc_trop_check>0)
        tc_trop_check_nums=labeled_tc_trop_check.values.flatten().tolist()
        tc_trop_check_nums=[x for x in tc_trop_check_nums if x>0]
        tc_trop_check_nums=list(set(tc_trop_check_nums))
        
        labeled_tc_nontrop_check=labeled_select.where(tc_nontrop_select==1)
        labeled_tc_nontrop_check=labeled_tc_nontrop_check.where(labeled_tc_nontrop_check>0)
        tc_nontrop_check_nums=labeled_tc_nontrop_check.values.flatten().tolist()
        tc_nontrop_check_nums=[x for x in tc_nontrop_check_nums if x>0]
        tc_nontrop_check_nums=list(set(tc_nontrop_check_nums))
        
        
        rain_area_eivt=four_labeled_rain.sel(time=date)
        rain_area_eivt=rain_area_eivt.where(rain_area_eivt!=0)
        rain_area_num=rain_area_eivt.max().values
        rain_area_num2=rain_area_eivt.min().values
        
        labeled_ar_check=labeled_select.where(ar_select==1)
        labeled_ar_check=labeled_ar_check.where(labeled_ar_check>0)
        ar_check_nums=labeled_ar_check.values.flatten().tolist()
        ar_check_nums=[x for x in ar_check_nums if x>0]
        ar_check_nums=list(set(ar_check_nums))
        
        if rain_area_num!=rain_area_num2:
            print('MULTIPLE EP IVT REGIONS')
            rain_area_num_values=rain_area_eivt.values.flatten().tolist()
            rain_area_num_values=[x for x in rain_area_num_values if x>0]
            rain_area_num_values=list(set(rain_area_num_values))
            rain_tc_trop_intersect=[x for x in rain_area_num_values if x in tc_trop_check_nums]
            if len(rain_tc_trop_intersect)>0:
                tc_trop_linked_ivt='yes'
            ar_tc_trop_intersect=[x for x in rain_tc_trop_intersect if x in ar_check_nums]#extreme IVT over rain region needs to be part of AR AND of TC regions
            if len(ar_tc_trop_intersect)>0:
                tc_trop_linked_ar='yes'
                
            rain_tc_nontrop_intersect=[x for x in rain_area_num_values if x in tc_nontrop_check_nums]
            if len(rain_tc_nontrop_intersect)>0:
                tc_nontrop_linked_ivt='yes'
            ar_tc_nontrop_intersect=[x for x in rain_tc_nontrop_intersect if x in ar_check_nums]#extreme IVT over rain region needs to be part of AR AND of TC regions
            if len(ar_tc_nontrop_intersect)>0:
                tc_trop_linked_ar='yes'
                
        else:
            if rain_area_num in tc_trop_check_nums:
                tc_trop_linked_ivt='yes'
            if rain_area_num in ar_check_nums and rain_area_num in tc_trop_check_nums:
                tc_trop_linked_ar='yes'
                
            if rain_area_num in tc_nontrop_check_nums:
                tc_nontrop_linked_ivt='yes'
            if rain_area_num in ar_check_nums and rain_area_num in tc_nontrop_check_nums:
                tc_nontrop_linked_ar='yes'

        print(ar_check_nums)
        print(tc_trop_check_nums)
        print('TC_NONTROP_LINKED_AR')
        print(tc_nontrop_linked_ar)
        print('TC_NONTROP_LINKED_IVT')
        print(tc_nontrop_linked_ivt)
        
        check_lat_tc=lat_tc.where(years_tc==year,drop=True)
        check_lat_tc=check_lat_tc.where(months_tc==month,drop=True)
        check_lat_tc=check_lat_tc.where(days_tc==day,drop=True)
        check_lat_tc=check_lat_tc.where(hours_tc==hour,drop=True)
        #TS,SS,EX,ET,
        check_lat_tc_trop=check_lat_tc.where(nature_tc==1,drop=True)
        check_lat_tc_nontrop=check_lat_tc.where(nature_tc!=1,drop=True)
        
        check_lat_tc_trop_list=[]
        for i in range(check_lat_tc_trop.storm.size):
            check_lat_tc_trop_values=check_lat_tc_trop[i,:].values.tolist()
            check_lat_tc_trop_values=[x for x in check_lat_tc_trop_values if x>0]
            if len(check_lat_tc_trop_values)>0:
                check_lat_tc_trop_value=check_lat_tc_trop_values[0]
                check_lat_tc_trop_list.append(check_lat_tc_trop_value)
                
        check_lat_tc_nontrop_list=[]
        for i in range(check_lat_tc_nontrop.storm.size):
            check_lat_tc_nontrop_values=check_lat_tc_nontrop[i,:].values.tolist()
            check_lat_tc_nontrop_values=[x for x in check_lat_tc_nontrop_values if x>0]
            if len(check_lat_tc_nontrop_values)>0:
                check_lat_tc_nontrop_value=check_lat_tc_nontrop_values[0]
                check_lat_tc_nontrop_list.append(check_lat_tc_nontrop_value)
        
        check_lon_tc=lon_tc.where(years_tc==year,drop=True)
        check_lon_tc=check_lon_tc.where(months_tc==month,drop=True)
        check_lon_tc=check_lon_tc.where(days_tc==day,drop=True)
        check_lon_tc=check_lon_tc.where(hours_tc==hour,drop=True)
        check_lon_tc_trop=check_lon_tc.where(nature_tc==1,drop=True)
        check_lon_tc_nontrop=check_lon_tc.where(nature_tc!=1,drop=True)
        
        check_lon_tc_trop_list=[]
        for i in range(check_lon_tc_trop.storm.size):
            check_lon_tc_trop_values=check_lon_tc_trop[i,:].values.tolist()
            check_lon_tc_trop_values=[x for x in check_lon_tc_trop_values if x>-1000]
            if len(check_lon_tc_trop_values)>0:
                check_lon_tc_trop_value=check_lon_tc_trop_values[0]
                check_lon_tc_trop_list.append(check_lon_tc_trop_value)
                
        check_lon_tc_nontrop_list=[]
        for i in range(check_lon_tc_nontrop.storm.size):
            check_lon_tc_nontrop_values=check_lon_tc_nontrop[i,:].values.tolist()
            check_lon_tc_nontrop_values=[x for x in check_lon_tc_nontrop_values if x>-1000]
            if len(check_lon_tc_nontrop_values)>0:
                check_lon_tc_nontrop_value=check_lon_tc_nontrop_values[0]
                check_lon_tc_nontrop_list.append(check_lon_tc_nontrop_value)
        
        if len(check_lat_tc_trop_list)>0 and len(check_lon_tc_trop_list)>0:
            min_distance_tc=2000.
            for i in range(len(check_lat_tc_trop_list)):
                lat_tc_single=check_lat_tc_trop_list[i]
                lon_tc_single=check_lon_tc_trop_list[i]
                if 18<=lat_tc_single<=29.5 and 116<=lon_tc_single<=126:
                    tc_trop_lat_list_day.append(lat_tc_single)
                    tc_trop_lon_list_day.append(lon_tc_single)
        if len(check_lat_tc_nontrop_list)>0 and len(check_lon_tc_nontrop_list)>0:
            min_distance_tc=2000.
            for i in range(len(check_lat_tc_nontrop_list)):
                lat_tc_single=check_lat_tc_nontrop_list[i]
                lon_tc_single=check_lon_tc_nontrop_list[i]
                if 18<=lat_tc_single<=29.5 and 116<=lon_tc_single<=126:
                    tc_nontrop_lat_list_day.append(lat_tc_single)
                    tc_nontrop_lon_list_day.append(lon_tc_single)
                  
    
    for date in date_range:
        print(date)
        date=pd.to_datetime(date)
        
        ivt_mag_current=ivt_mag.sel(time=date)
        ivtx_current=ivtx.sel(time=date)
        ivty_current=ivty.sel(time=date)
        print(u_850)
        u_850_select=u_850.sel(time=date)
        v_850_select=v_850.sel(time=date)
        slp_select=slp.sel(time=date)

        ivt_extreme=ivt_mag_current.where(ivt_mag_current>=ivt_threshold)
        print(ivt_extreme)
        ivt_250=ivt_mag_current.where(ivt_mag_current>=250.)
        ivt_ones=ivt_extreme/ivt_extreme
        ivt_ones=ivt_ones.fillna(0)
        
        ar_select=ar_date_four.sel(time=date)

        year=date.year
        month=date.month
        day=date.day
        hour=date.hour
        
        fig = plt.figure(figsize=(26,15))
        ax=plt.subplot(1,1,1,projection=ccrs.PlateCarree())
        
        check_lat_tc=lat_tc.where(years_tc==year,drop=True)
        check_lat_tc=check_lat_tc.where(months_tc==month,drop=True)
        check_lat_tc=check_lat_tc.where(days_tc==day,drop=True)
        check_lat_tc=check_lat_tc.where(hours_tc==hour,drop=True)
        #TS,SS,EX,ET,
        check_lat_tc_trop=check_lat_tc.where(nature_tc==1,drop=True)
        check_lat_tc_nontrop=check_lat_tc.where(nature_tc!=1,drop=True)
        
        check_lat_tc_trop_list=[]
        for i in range(check_lat_tc_trop.storm.size):
            check_lat_tc_trop_values=check_lat_tc_trop[i,:].values.tolist()
            check_lat_tc_trop_values=[x for x in check_lat_tc_trop_values if x>0]
            if len(check_lat_tc_trop_values)>0:
                check_lat_tc_trop_value=check_lat_tc_trop_values[0]
                check_lat_tc_trop_list.append(check_lat_tc_trop_value)
                
        check_lat_tc_nontrop_list=[]
        for i in range(check_lat_tc_nontrop.storm.size):
            check_lat_tc_nontrop_values=check_lat_tc_nontrop[i,:].values.tolist()
            check_lat_tc_nontrop_values=[x for x in check_lat_tc_nontrop_values if x>0]
            if len(check_lat_tc_nontrop_values)>0:
                check_lat_tc_nontrop_value=check_lat_tc_nontrop_values[0]
                check_lat_tc_nontrop_list.append(check_lat_tc_nontrop_value)
        
        check_lon_tc=lon_tc.where(years_tc==year,drop=True)
        check_lon_tc=check_lon_tc.where(months_tc==month,drop=True)
        check_lon_tc=check_lon_tc.where(days_tc==day,drop=True)
        check_lon_tc=check_lon_tc.where(hours_tc==hour,drop=True)
        check_lon_tc_trop=check_lon_tc.where(nature_tc==1,drop=True)
        check_lon_tc_nontrop=check_lon_tc.where(nature_tc!=1,drop=True)
        
        check_lon_tc_trop_list=[]
        for i in range(check_lon_tc_trop.storm.size):
            check_lon_tc_trop_values=check_lon_tc_trop[i,:].values.tolist()
            check_lon_tc_trop_values=[x for x in check_lon_tc_trop_values if x>-1000]
            if len(check_lon_tc_trop_values)>0:
                check_lon_tc_trop_value=check_lon_tc_trop_values[0]
                check_lon_tc_trop_list.append(check_lon_tc_trop_value)
                
        check_lon_tc_nontrop_list=[]
        for i in range(check_lon_tc_nontrop.storm.size):
            check_lon_tc_nontrop_values=check_lon_tc_nontrop[i,:].values.tolist()
            check_lon_tc_nontrop_values=[x for x in check_lon_tc_nontrop_values if x>-1000]
            if len(check_lon_tc_nontrop_values)>0:
                check_lon_tc_nontrop_value=check_lon_tc_nontrop_values[0]
                check_lon_tc_nontrop_list.append(check_lon_tc_nontrop_value)
        
        if len(check_lat_tc_trop_list)>0 and len(check_lon_tc_trop_list)>0:
            for i in range(len(check_lat_tc_trop_list)):
                lat_tc_single=check_lat_tc_trop_list[i]
                lon_tc_single=check_lon_tc_trop_list[i]
                ax.plot(lon_tc_single,lat_tc_single,marker='*',color='r',transform=ccrs.PlateCarree(),markersize=24,zorder=41)
                if 18<=lat_tc_single<=29.5 and 116<=lon_tc_single<=126:
                    tc_trop_lat_list.append(lat_tc_single)
                    tc_trop_lon_list.append(lon_tc_single)
        if len(check_lat_tc_nontrop_list)>0 and len(check_lon_tc_nontrop_list)>0:
            for i in range(len(check_lat_tc_nontrop_list)):
                lat_tc_single=check_lat_tc_nontrop_list[i]
                lon_tc_single=check_lon_tc_nontrop_list[i]
                ax.plot(lon_tc_single,lat_tc_single,marker='*',color='saddlebrown',transform=ccrs.PlateCarree(),markersize=24,zorder=41)
                if 18<=lat_tc_single<=29.5 and 116<=lon_tc_single<=126:
                    tc_nontrop_lat_list.append(lat_tc_single)
                    tc_nontrop_lon_list.append(lon_tc_single)
                            
        ax.coastlines(resolution='10m')
        ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
        ax.add_feature(cfeature.LAKES.with_scale('50m'))
        countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
        ax.add_feature(countries)
        ax.set_extent([85,160,5,50],crs=ccrs.PlateCarree())

        # *must* call draw in order to get the axis boundary used to add ticks:
        #fig.canvas.draw()

        # Define gridline locations and draw the lines using cartopy's built-in gridliner:
        xticks = [85,90,95,100,105,110,115,120,125,130,135,140,145,150,155,160]
        yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
        ax.tick_params(labelsize=20)
        g1=ax.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,alpha=0)
        g1.xformatter=LONGITUDE_FORMATTER
        g1.yformatter=LATITUDE_FORMATTER
        g1.xlabel_style={'size':22,'color':'k'}
        g1.ylabel_style={'size':22,'color':'k'}
        g1.top_labels=False
        g1.right_labels=False

        #cax=ax.scatter(lons_extreme_list,lats_extreme_list,s=150,c=obs_extreme_list,transform=ccrs.PlateCarree(),vmin=0,vmax=120,cmap=plt.cm.RdYlGn_r,zorder=10)
        cax0=ax.contourf(p_select_extreme.lon,p_select_extreme.lat,p_select_extreme,levels=np.arange(0,240,5),extend='max',cmap=plt.cm.Greens)
        cax1=ax.contourf(ivt_extreme.longitude,ivt_extreme.latitude,ivt_250,levels=np.arange(250,1040,50),extend='both',transform=ccrs.PlateCarree(),cmap=plt.cm.Greys,alpha=0.5)
        ax.set_title(str(hour)+'Z '+month_dict[str(date.month)]+' '+str(date.day)+', '+str(date.year),fontsize=30)
        cbar=plt.colorbar(cax1,pad=0,fraction=0.043)
        cbar.set_label('kg/m/s',fontsize=20,rotation=90)
        cbar.ax.tick_params(labelsize=20)

        
        if 1==0:
            for i in range(len(extreme_ivt_lat_list)):
                lat=extreme_ivt_lat_list[i]
                lon=extreme_ivt_lon_list[i]
                if i==0:
                    ax.plot(lon,lat,marker='o',color='k',markersize=25,mew=2.0,markerfacecolor='none',transform=ccrs.PlateCarree(),label='Extreme IVT present')
                else:
                    ax.plot(lon,lat,marker='o',color='k',markersize=25,mew=2.0,markerfacecolor='none',transform=ccrs.PlateCarree())

            for i in range(len(ar_lat_list)):
                lat=ar_lat_list[i]
                lon=ar_lon_list[i]
                if i==0:
                    ax.plot(lon,lat,marker='o',color='b',markersize=25,mew=2.0,markerfacecolor='none',transform=ccrs.PlateCarree(),label='Extreme IVT present')
                else:
                    ax.plot(lon,lat,marker='o',color='b',markersize=25,mew=2.0,markerfacecolor='none',transform=ccrs.PlateCarree())

        c=ax.contour(slp_select.longitude,slp_select.latitude,slp_select,transform=ccrs.PlateCarree(),colors='k',levels=np.arange(960,1040,4))
        plt.clabel(c,fmt='%.00f',fontsize=15)
        X=4
        q=ax.quiver(u_850_select.longitude[0::X].values,u_850_select.latitude[0::X].values,u_850_select[0::X,0::X].values,v_850_select[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=40,width=0.015,alpha=0.5)
        ax.quiverkey(q,0.94,1.01,10,'10 m/s',fontproperties={'size':13})

        ax.set_title(str(hour)+'Z '+month_dict[str(date.month)]+' '+str(date.day)+', '+str(date.year),fontsize=30)
        
        #for i in range(len(points_closest_list)):
        #    if i==0:
        #       ax.plot(points_closest_list[i][0],points_closest_list[i][1],'ko',transform=ccrs.PlateCarree(),markersize=12,label='Closest ERA5 grid point')
        #    else:
        #        ax.plot(points_closest_list[i][0],points_closest_list[i][1],'ko',transform=ccrs.PlateCarree(),markersize=12)
        
        if 1==0:
            for i in range(len(extreme_ivt_lat_list)):
                lat=extreme_ivt_lat_list[i]
                lon=extreme_ivt_lon_list[i]
                if i==0:
                    ax.plot(lon,lat,marker='o',color='m',markersize=20,mew=2.0,markerfacecolor='none',transform=ccrs.PlateCarree(),label='Extreme IVT with EP')
                else:
                    ax.plot(lon,lat,marker='o',color='m',markersize=20,mew=2.0,markerfacecolor='none',transform=ccrs.PlateCarree())
        if 1==0:
            for i in range(len(ar_lat_list)):
                lat=ar_lat_list[i]
                lon=ar_lon_list[i]
                if i==0:
                    ax.plot(lon,lat,marker='o',color='b',markersize=20,mew=2.0,markerfacecolor='none',transform=ccrs.PlateCarree(),label='AR with EP')
                else:
                    ax.plot(lon,lat,marker='o',color='b',markersize=20,mew=2.0,markerfacecolor='none',transform=ccrs.PlateCarree())
        if 1==0:
            for i in range(len(tc_trop_lat_list_day)):
                lat=tc_trop_lat_list_day[i]
                lon=tc_trop_lon_list_day[i]
                if i==0:
                    ax.plot(lon,lat,marker='o',color='indigo',markersize=22,mew=2.0,markerfacecolor='none',transform=ccrs.PlateCarree(),zorder=40,label='TC (500km) with EP')
                else:
                    ax.plot(lon,lat,marker='o',color='indigo',markersize=22,mew=2.0,markerfacecolor='none',transform=ccrs.PlateCarree(),zorder=40)
        cax21=ax.contourf(ivt_extreme.longitude,ivt_extreme.latitude,ivt_250.where(ar_select==1),levels=np.arange(250,1040,50),extend='both',transform=ccrs.PlateCarree(),cmap=plt.cm.Greys,alpha=0.5,label='Identified AR')
        cax2=ax.contourf(ivt_extreme.longitude,ivt_extreme.latitude,ar_select,[0,1],colors='b',transform=ccrs.PlateCarree(),hatches=[None,'.'],alpha=0.1,label='Identified AR',zorder=20)
        
        mpl.rcParams['hatch.linewidth']=0.01
        if 1==0:
            for i in range(ar_select.latitude.size):
                for j in range(ar_select.longitude.size):
                    if ar_select[i,j]==1:
                        ax.plot(lons_full[j],lats_full[i],'bo',transform=ccrs.PlateCarree(),markersize=0.5,label='Identified AR')
        if len(tc_trop_lat_list_day)>0:
            #plt.text(0,1,'Class: Direct TC',fontsize=12)
            if len(ar_lat_list)>0:
                plt.annotate('Class: TC-AR combination', xy=(0, 1.02), xycoords='axes fraction',fontsize=24)
            else:
                plt.annotate('Class: Direct TC', xy=(0, 1.02), xycoords='axes fraction',fontsize=24)
        elif len(ar_lat_list)>0:
            if tc_trop_linked_ar=='yes':
                plt.annotate('Class: TC-linked AR', xy=(0, 1.02), xycoords='axes fraction',fontsize=24)
            elif tc_nontrop_linked_ar=='yes':
                if len(tc_nontrop_lat_list_day)>0:
                    plt.annotate('Class: TC-remnant AR combination', xy=(0, 1.02), xycoords='axes fraction',fontsize=24)
                else:
                    plt.annotate('Class: TC-remnant linked AR', xy=(0, 1.02), xycoords='axes fraction',fontsize=24)
            else:
                plt.annotate('Class: Direct AR', xy=(0, 1.02), xycoords='axes fraction',fontsize=24)
        elif len(tc_nontrop_lat_list_day)>0:
            plt.annotate('Class: TC remnants', xy=(0, 1.02), xycoords='axes fraction',fontsize=24)
        elif len(extreme_ivt_value_list)>0:
            if tc_trop_linked_ivt=='yes':
                plt.annotate('Class: TC-linked IVT', xy=(0, 1.02), xycoords='axes fraction',fontsize=24)
            elif tc_nontrop_linked_ivt=='yes':
                plt.annotate('Class: TC-remnant IVT', xy=(0, 1.02), xycoords='axes fraction',fontsize=24)
            else:
                plt.annotate('Class: Pure Extreme IVT', xy=(0, 1.02), xycoords='axes fraction',fontsize=24)
        else:
            plt.annotate('Class: unspecified', xy=(0, 1.02), xycoords='axes fraction',fontsize=24)
            
        left, bottom, width, height = [0.14, 0.63, 0.12, 0.24]
        ax2 = fig.add_axes([left, bottom, width, height],projection=ccrs.PlateCarree())
        cax5=ax2.contourf(p_select_extreme.lon,p_select_extreme.lat,p_select_extreme,levels=np.arange(0,240,5),extend='max',cmap=plt.cm.Greens)
        ax2.add_feature(cfeature.COASTLINE.with_scale('10m'))
        ax2.set_xlim(119.7,122.3)
        ax2.set_ylim(21.8,25.5)
        #plt.legend(loc='upper left',fontsize=20)
        #plt.show()
        fig.savefig(dir+'taiwan_event_ar_detection_var99_'+str(count_fig)+'.png')
        count_fig=count_fig+1
        plt.close()
    
    if len(tc_trop_lat_list_day)>0:
        if len(ar_lat_list)>0:
            tc_ar_combo_yesno.append(1)
            tc_yesno.append(0)
            other_yesno.append(0)
            ar_yesno.append(0)
            ivt_yesno.append(0)
            tc_linked_ivt_yesno.append(0)
            tc_linked_ar_yesno.append(0)
            ivt_tc_remnants_yesno.append(0)
            ar_tc_remnant_combo_yesno.append(0)
            tc_remnant_linked_ar_yesno.append(0)
            tc_remnants_yesno.append(0)
        else:
            tc_ar_combo_yesno.append(0)
            tc_yesno.append(1)
            other_yesno.append(0)
            ar_yesno.append(0)
            ivt_yesno.append(0)
            tc_linked_ivt_yesno.append(0)
            tc_linked_ar_yesno.append(0)
            ivt_tc_remnants_yesno.append(0)
            ar_tc_remnant_combo_yesno.append(0)
            tc_remnant_linked_ar_yesno.append(0)
            tc_remnants_yesno.append(0)
    elif len(ar_lat_list)>0:
        other_yesno.append(0)
        tc_yesno.append(0)
        tc_linked_ivt_yesno.append(0)
        ivt_yesno.append(0)
        tc_ar_combo_yesno.append(0)
        if tc_trop_linked_ar=='yes':
            tc_linked_ar_yesno.append(1)
            ar_yesno.append(0)
            ivt_tc_remnants_yesno.append(0)
            ar_tc_remnant_combo_yesno.append(0)
            tc_remnant_linked_ar_yesno.append(0)
            tc_remnants_yesno.append(0)
        elif tc_nontrop_linked_ar=='yes':
            if len(tc_nontrop_lat_list_day)>0:
                ivt_tc_remnants_yesno.append(0)
                ar_tc_remnant_combo_yesno.append(1)
                tc_remnant_linked_ar_yesno.append(0)
                tc_remnants_yesno.append(0)
                ar_yesno.append(0)
                tc_linked_ar_yesno.append(0)
            else:
                ivt_tc_remnants_yesno.append(0)
                ar_tc_remnant_combo_yesno.append(0)
                tc_remnant_linked_ar_yesno.append(1)
                tc_remnants_yesno.append(0)
                ar_yesno.append(0)
                tc_linked_ar_yesno.append(0)
        elif len(tc_nontrop_lat_list_day)>0:
            ivt_tc_remnants_yesno.append(0)
            ar_tc_remnant_combo_yesno.append(1)
            tc_remnant_linked_ar_yesno.append(0)
            tc_remnants_yesno.append(0)
            ar_yesno.append(0)
            tc_linked_ar_yesno.append(0)
        else:
            ar_yesno.append(1)
            tc_linked_ar_yesno.append(0)
            ivt_tc_remnants_yesno.append(0)
            ar_tc_remnant_combo_yesno.append(0)
            tc_remnant_linked_ar_yesno.append(0)
            tc_remnants_yesno.append(0)
    elif len(tc_nontrop_lat_list_day)>0:
        ivt_tc_remnants_yesno.append(0)
        ar_tc_remnant_combo_yesno.append(0)
        tc_remnant_linked_ar_yesno.append(0)
        tc_remnants_yesno.append(1)
        tc_ar_combo_yesno.append(0)
        ar_yesno.append(0)
        tc_yesno.append(0)
        ivt_yesno.append(0)
        tc_linked_ar_yesno.append(0)
        tc_linked_ivt_yesno.append(0)
        other_yesno.append(0)
    elif len(extreme_ivt_value_list)>0:
        other_yesno.append(0)
        ar_yesno.append(0)
        tc_yesno.append(0)
        tc_linked_ar_yesno.append(0)
        tc_ar_combo_yesno.append(0)
        if tc_trop_linked_ivt=='yes':#would probably be included in tc_linked_ivt anyway
            tc_linked_ivt_yesno.append(1)
            ivt_yesno.append(0)
            ivt_tc_remnants_yesno.append(0)
            ar_tc_remnant_combo_yesno.append(0)
            tc_remnant_linked_ar_yesno.append(0)
            tc_remnants_yesno.append(0)
        elif tc_nontrop_linked_ivt=='yes':
            ivt_tc_remnants_yesno.append(1)
            ar_tc_remnant_combo_yesno.append(0)
            tc_remnant_linked_ar_yesno.append(0)
            tc_remnants_yesno.append(0)
            tc_linked_ivt_yesno.append(0)
            ivt_yesno.append(0)
        else:
            tc_linked_ivt_yesno.append(0)
            ivt_yesno.append(1)
            ivt_tc_remnants_yesno.append(0)
            ar_tc_remnant_combo_yesno.append(0)
            tc_remnant_linked_ar_yesno.append(0)
            tc_remnants_yesno.append(0)
    else:
        other_yesno.append(1)
        ar_yesno.append(0)
        tc_yesno.append(0)
        ivt_yesno.append(0)
        tc_linked_ivt_yesno.append(0)
        tc_linked_ar_yesno.append(0)
        ivt_tc_remnants_yesno.append(0)
        ar_tc_remnant_combo_yesno.append(0)
        tc_remnants_yesno.append(0)
        tc_ar_combo_yesno.append(0)
        tc_remnant_linked_ar_yesno.append(0)
    
        
dk=xr.Dataset()
dk['ar_yesno']=(('time'),ar_yesno)
dk['ivt_yesno']=(('time'),ivt_yesno)
dk['tc_yesno']=(('time'),tc_yesno)
dk['other_yesno']=(('time'),other_yesno)
dk['tc_linked_ar_yesno']=(('time'),tc_linked_ar_yesno)
dk['tc_linked_ivt_yesno']=(('time'),tc_linked_ivt_yesno)
dk['ivt_tc_remnants_yesno']=(('time'),ivt_tc_remnants_yesno)#TC remnant-linked IVT
dk['ar_tc_remnant_combo_yesno']=(('time'),ar_tc_remnant_combo_yesno)
dk['tc_remnant_linked_ar_yesno']=(('time'),tc_remnant_linked_ar_yesno)
dk['tc_remnants_yesno']=(('time'),tc_remnants_yesno)
dk['tc_ar_combo_yesno']=(('time'),tc_ar_combo_yesno)
#dk['artc_yesno']=(('time'),artc_yesno)
dk.coords['time']=ar.time[0::4]
#dk.coords['latitude']=ar.latitude
#dk.coords['longitude']=ar.longitude

try:
    os.remove(dir+'taiwan_ep_days_stats_var99_newar.nc')
except OSError:
    pass
dk.to_netcdf(dir+'taiwan_ep_days_stats_var99_newar.nc',mode='w',format='NETCDF4')